# Data cleaning

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Cleaning dataset 1

In [ ]:
df1 = pd.read_csv("Access_and_Use_of_Telemedicine_During_COVID-19.csv")
columns_to_keep = [col for col in df1.columns if col not in {'Suppression', 'Significant 1', 'Significant 2'}]
df1_filtered = df1[columns_to_keep]

df1_filtered = df1_filtered[(df1_filtered['Group'] != 'Total') & (df1_filtered['Subgroup'] != 'Total') & (df1_filtered['Response'] != 'Total')]

In [ ]:
columns_to_keep = [col for col in df1_filtered.columns if col not in {'Suppression', 'Significant 1', 'Significant 2'}]

df1_filtered = df1_filtered[columns_to_keep]

df1_clean = df1_filtered
df1_clean = df1_clean[df1_clean['Group'].isin({'Urbanization', 'Education', 'Race/Hispanic origin'})]

df1_clean.loc[df1_clean['Group'] == 'Education', 'Subgroup'] =  df1[df1['Group'] == 'Education']['Subgroup'].replace({
    'High school graduate or less': 'highschool or less',
    'Some college': 'college',
    "Bachelor's degree or above": 'bachelor or above'
})

df1_clean.loc[df1_clean['Group'] == 'Urbanization', 'Subgroup'] = df1[df1['Group'] == 'Urbanization']['Subgroup'].replace({
    'Metropolitan': 'Urban',
    'Non-Metropolitan': 'Rural'
})

In [ ]:
df1_final = df1_clean

total_sample_sizes = (
    df1_clean[df1_clean['Response'] == 'Total']
    .groupby(['Round', 'Indicator', 'Group', 'Subgroup'])['Sample Size']
    .sum()
    .reset_index()
)


total_sample_sizes = total_sample_sizes.rename(columns={'Sample Size': 'Total Sample Size'})

# merge the total sample sizes back into the original df
df1_clean = pd.merge(df1_clean, total_sample_sizes, on=['Round', 'Indicator', 'Group', 'Subgroup'], how='left')

# calculate "size"
df1_clean['size'] = (df1_clean['Percent'] * df1_clean['Total Sample Size']) / 100

# drop the temporary column
df1_clean.drop(columns=['Total Sample Size'], inplace=True)

In [ ]:
df1_clean.rename(columns={"Sample Size": "SampleSize"}, inplace=True)
df1_clean.rename(columns={"Standard Error": "StandardError"}, inplace=True)
df1_clean.rename(columns={"Group": "majorGroup"}, inplace=True)
df1_clean.rename(columns={"Subgroup": "SubGroup"}, inplace=True)
df1_clean.rename(columns={"size": "sizeValue"}, inplace=True)

df1_clean.replace({pd.NA: None, np.nan: None}, inplace=True)

In [ ]:
df1_final = df1_clean.pivot(
    index=['Round', 'Indicator', 'majorGroup', 'SubGroup', 'SampleSize', 'sizeValue'],
    columns='Response',
    values=['Percent', 'StandardError']
).reset_index()

# Flatten the column MultiIndex
df1_final.columns = [f"{col[0]}_{col[1]}" if col[1] else col[0] for col in df1_final.columns]

# Create Race and Urbanization columns
df1_final['Race'] = df1_final['SubGroup'].where(df1_final['majorGroup'] == 'Race/Hispanic origin')
df1_final['Urbanization'] = df1_final['SubGroup'].where(df1_final['majorGroup'] == 'Urbanization')
df1_final['Education'] = df1_final['SubGroup'].where(df1_final['majorGroup'] == 'Education')


# Drop redundant columns
df1_final.drop(['majorGroup', 'SubGroup'], axis=1, inplace=True)

In [ ]:
def clean_colnames(text):
  names = text.lower()
  names = names.split(' ')
  final_name = names[0]
  for i in names[1:]:
    final_name += '_'+i
  return final_name

df1_final.columns = list(map(clean_colnames, df1_final.columns))

urbanization_mapping = {
    "Rural": "Rural",
    "Urban": "Urban",
    np.nan: "Unknown"  # Map missing values to "Unknown"
}

race_mapping = {
    "Rural": "Rural",
    "Urban": "Urban",
    np.nan: "Other non-Hispanic"  # Map missing values to "Unknown"
}

education_mapping = {
    'bachelor or above':'bachelor or above',
    'college':'college',
    'highschool or less':'highschool or less',
    np.nan: "Unknown"
}
df1_final["urbanization"] = df1_final["urbanization"].replace(urbanization_mapping)
df1_final["race"] = df1_final["race"].map(race_mapping).fillna(df1_final["race"])
df1_final["education"] = df1_final["education"].map(education_mapping).fillna(df1_final["education"])

# drop useless rows
df1_final = df1_final[
    (df1_final['race'] != 'Other non-Hispanic') |
    (df1_final['education'] != 'Unknown') |
    (df1_final['urbanization'] != 'Unknown')
]

## Cleaning dataset 2

In [ ]:
df2 = pd.read_csv("TMEDTREND_PUBLIC_241126.csv")

df2_filtered = df2[df2['Year'].between(2020, 2022)]
df2_filtered = df2_filtered[(df2_filtered['Bene_Race_Desc'] != 'All') & (df2_filtered['Bene_RUCA_Desc'] != 'All')]

In [ ]:
# Step 2: Aggregate the data by relevant columns and sum numerical columns
df2_clean = (
    df2_filtered.groupby(['Bene_Race_Desc', 'Bene_RUCA_Desc', 'Bene_Mdcd_Mdcr_Enrl_Stus'])
    .agg({
        'Total_Bene_TH_Elig': 'sum',
        'Total_PartB_Enrl': 'sum',
        'Total_Bene_Telehealth': 'sum',
        'Pct_Telehealth': 'mean'  # Use mean for percentage columns
    })
    .reset_index()
)


df2_final = df2_clean

def col_replacer(x):
  if x == 'Bene_Race_Desc': return 'Race'
  elif x == 'Bene_RUCA_Desc': return 'Urbanization'
  elif x == 'Bene_Mdcd_Mdcr_Enrl_Stus': return 'Enrollment_Status'
  else: return x

cols = list(df2_clean.columns)
cols_transformed = list(map(col_replacer, cols))
df2_final.columns = cols_transformed

race_mapping = {
    "All": "Total",
    "Non-Hispanic White": "White non-Hispanic",
    "Black/African American": "Black non-Hispanic",
    "Other/Unknown": "Other non-Hispanic",
    "Hispanic": "Hispanic"
}

urban_mapping = {
    "Urban": "Urban",
    "Rural": "Rural"
}

# Apply mappings, keeping unmatched values
df2_final["Race"] = df2_final["Race"].map(race_mapping).fillna(df2_final["Race"])
df2_final["Urbanization"] = df2_final["Urbanization"].map(urban_mapping).fillna(df2_final["Urbanization"])
df2_final.replace({pd.NA: None, np.nan: None}, inplace=True)
df2_final.columns = list(map(clean_colnames, df2_final.columns))

In [ ]:
df1_final.to_csv('Access_and_telemedicine_use_COVID19_CLEAN.csv', index=False)
df2_final.to_csv('TIMETREND_PUBLIC_CLEAN.csv', index=False)

# SQL script generator


In [ ]:
df1 = pd.read_csv('Access_and_telemedicine_use_COVID19_CLEAN.csv')
df2 = pd.read_csv('TIMETREND_PUBLIC_CLEAN.csv')

In [ ]:
def quote(val):
    if pd.isna(val):
        return 'NULL'
    safe_val = str(val).replace('&', 'and').replace("'", "''")
    return f"'{safe_val}'"

In [ ]:
sql_filename = "teleHealthData.sql"

with open(sql_filename, "w") as f:
    f.write("""
DROP TABLE telemedicineprovider PURGE;
DROP TABLE beneficiarydata PURGE;

-- Create table for BeneficiaryData data
CREATE TABLE beneficiarydata (
  race VARCHAR(255),
  urbanization VARCHAR(255),
  enrollment_status VARCHAR(255),
  total_bene_th_elig FLOAT,
  total_partb_enrl FLOAT,
  total_bene_telehealth FLOAT,
  pct_telehealth FLOAT,
  PRIMARY KEY (
    race, urbanization
  )
);

-- Create table for TelemedicineProvider data
CREATE TABLE telemedicineprovider (
  round INT,
  indicator VARCHAR(255),
  race VARCHAR(255),
  urbanization VARCHAR(255),
  education VARCHAR(255),
  samplesize FLOAT,
  sizevalue FLOAT,
  percent_do_not_know FLOAT,
  percent_no FLOAT,
  percent_no_telemedicine_available FLOAT,
  percent_no_usual_place_of_care FLOAT,
  percent_yes FLOAT,
  standarderror_do_not_know FLOAT,
  standarderror_no FLOAT,
  standarderror_no_telemedicine_available FLOAT,
  standarderror_no_usual_place_of_care FLOAT,
  standarderror_yes FLOAT,
  PRIMARY KEY (round, indicator, race, urbanization, education),
  FOREIGN KEY (race, urbanization) REFERENCES beneficiarydata(race, urbanization));

""")

    f.write("-- SQL Script for Bulk Insert into Oracle DB\n\n")

    # Insert statements for beneficiarydata
    f.write("INSERT ALL\n")
    for _, row in df2.iterrows():
        f.write("    INTO beneficiarydata (race, urbanization, enrollment_status, total_bene_th_elig, total_partb_enrl, total_bene_telehealth, pct_telehealth)\n")
        f.write(f"    VALUES ({quote(row['race'])}, {quote(row['urbanization'])}, {quote(row['enrollment_status'])}, "
                f"{'NULL' if pd.isna(row['total_bene_th_elig']) else float(row['total_bene_th_elig'])}, "
                f"{'NULL' if pd.isna(row['total_partb_enrl']) else float(row['total_partb_enrl'])}, "
                f"{'NULL' if pd.isna(row['total_bene_telehealth']) else float(row['total_bene_telehealth'])}, "
                f"{'NULL' if pd.isna(row['pct_telehealth']) else float(row['pct_telehealth'])})\n")
    f.write("SELECT 1 FROM DUAL;\n")

    # Insert statements for telemedicineprovider
    f.write("INSERT ALL\n")
    for _, row in df1.iterrows():
        f.write("    INTO telemedicineprovider (round, indicator, race, urbanization, education, samplesize, sizevalue,\n")
        f.write("         percent_do_not_know, percent_no, percent_no_telemedicine_available, percent_no_usual_place_of_care, percent_yes,\n")
        f.write("         standarderror_do_not_know, standarderror_no, standarderror_no_telemedicine_available, standarderror_no_usual_place_of_care, standarderror_yes)\n")
        f.write(f"    VALUES ({'NULL' if pd.isna(row['round']) else int(row['round'])}, "
                f"{quote(row['indicator'])}, {quote(row['race'])}, {quote(row['urbanization'])}, {quote(row['education'])}, "
                f"{'NULL' if pd.isna(row['samplesize']) else float(row['samplesize'])}, "
                f"{'NULL' if pd.isna(row['sizevalue']) else float(row['sizevalue'])}, "
                f"{'NULL' if pd.isna(row['percent_do_not_know']) else float(row['percent_do_not_know'])}, "
                f"{'NULL' if pd.isna(row['percent_no']) else float(row['percent_no'])}, "
                f"{'NULL' if pd.isna(row['percent_no_telemedicine_available']) else float(row['percent_no_telemedicine_available'])}, "
                f"{'NULL' if pd.isna(row['percent_no_usual_place_of_care']) else float(row['percent_no_usual_place_of_care'])}, "
                f"{'NULL' if pd.isna(row['percent_yes']) else float(row['percent_yes'])}, "
                f"{'NULL' if pd.isna(row['standarderror_do_not_know']) else float(row['standarderror_do_not_know'])}, "
                f"{'NULL' if pd.isna(row['standarderror_no']) else float(row['standarderror_no'])}, "
                f"{'NULL' if pd.isna(row['standarderror_no_telemedicine_available']) else float(row['standarderror_no_telemedicine_available'])}, "
                f"{'NULL' if pd.isna(row['standarderror_no_usual_place_of_care']) else float(row['standarderror_no_usual_place_of_care'])}, "
                f"{'NULL' if pd.isna(row['standarderror_yes']) else float(row['standarderror_yes'])})\n")
    f.write("SELECT 1 FROM DUAL;\n\n")

print("SQL script generated")

SQL script generated
